In [1]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv
from stable_baselines3 import SAC, HerReplayBuffer
from environments import *

import torch
import numpy as np
import gym
import imageio

c:\Users\ssuro\anaconda3\envs\torch\lib\site-packages\torch\utils\tensorboard\__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


In [2]:
env_id = 'G1ControlEnv-v1'
video_folder = 'renders'
video_length = 200

In [3]:
models = {
    "g1-control-direction": "trained_models/g1-control/direction",
    "g1-control-orientation": "trained_models/g1-control/orientation",
    "g1-control-direction+orientation": "trained_models/g1-control/direction+orientation",
}

In [4]:
import cv2
import scipy.misc
import os

import pillow_avif  # Have to import this before importing PIL
from PIL import Image


In [5]:
for name, model_path in models.items():

    if name == 'g1-control-direction':
        env = gym.make(env_id, mode='direction')
    
    if name == 'g1-control-orientation':
        env = gym.make(env_id, mode='orientation')
    
    if name == 'g1-control-direction+orientation':
        env = gym.make(env_id, mode='direction+orientation')

    model = SAC.load(model_path)

    for j, task in enumerate(env.sample_tasks(3)):
        env.reset_task(task)

        obs = env.reset()
        img = env.render(mode='rgb_array')

        images = []

        for i in range(video_length + 1):
            img = cv2.cvtColor(img.copy(), cv2.COLOR_RGBA2BGRA)
            origin = (240, 240)
            dest = (
                int(100 * np.cos(env.task['orientation'])) + 240, 
                int(100 * np.sin(env.task['orientation'])) + 240
            )

            # Draw orientation arrow
            if name == 'g1-control-orientation' or name == 'g1-control-direction+orientation':

                img = cv2.arrowedLine(img.copy(), origin, dest, (0, 255, 0), 9)

                (ax, ay, az) = env.get_euler_angles()

                dest = (
                    int(100 * np.cos(az)) + 240, 
                    int(100 * np.sin(az)) + 240
                )
                
                img = cv2.arrowedLine(img.copy(), origin, dest, (255, 0, 0), 9)


            if name == 'g1-control-direction+orientation' or name == 'g1-control-direction':

                dest = (
                    int(50 * np.cos(-env.task['direction'])) + 240, 
                    int(50 * np.sin(-env.task['direction'])) + 240
                )
                
                img = cv2.arrowedLine(img.copy(), origin, dest, (255, 255, 0), 9)

            images.append(img)
            action = model.predict(obs)
            
            obs, _, done, info = env.step(action[0])      
            if done: break

            img = env.render(mode='rgb_array')

        imageio.mimsave(
        'renders/' + name + '_' + str(j) + '.gif',
            [np.array(img) for i, img in enumerate(images) if i%4 == 0], 
            fps=29,
        )

        image = Image.open('renders/' + name + '_' + str(j) + '.gif')
        image.save('renders/' + name + '_' + str(j) + '.avif', quality=50, save_all=True, fps=29)
        image.close()
        os.remove('renders/' + name + '_' + str(j) + '.gif')

    env.close()

c:\Users\ssuro\anaconda3\envs\torch\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\ssuro\anaconda3\envs\torch\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\ssuro\anaconda3\envs\torch\lib\site-packages\gym\core.py:51: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
